# Defining eye models

Eye models provide a backend-independent specification of the optical eye model.
They can be created using the {py:class}`visisipy.models.EyeModel` class:

In [1]:
import visisipy

model = visisipy.EyeModel()

Eye models consist of two parts: the geometry of the eye and the material model.
By default, Visisipy uses the geometry and material properties from the  Navarro schematic eye
[](doi:10.1364/JOSAA.16.001881).

## Eye geometry

Geometries are defined in {py:mod}`visisipy.models.geometry`.
Apart from base classes that can be used to define your own geometry, this module also provides a number of predefined
geometries, such as the {py:class}`NavarroGeometry <visisipy.models.geometry.NavarroGeometry>`.
Geometry objects contain surface definitions for all surfaces in the eye model.
A fully customized geometry can be specified like this:

In [2]:
from visisipy.models.geometry import StandardSurface, Stop

custom_geometry = visisipy.EyeGeometry(
    cornea_front=StandardSurface(radius=7.72, asphericity=-0.26, thickness=0.55),
    cornea_back=StandardSurface(radius=6.50, asphericity=0, thickness=3.05),
    pupil=Stop(semi_diameter=1.348),
    lens_front=StandardSurface(radius=10.2, asphericity=-3.1316, thickness=4.0),
    lens_back=StandardSurface(radius=-6.0, asphericity=-1, thickness=16.3203),
    retina=StandardSurface(radius=-12.0, asphericity=0),
)

It is also possible to modify the predefined geometries. Complete surfaces can be replaced via the model constructor, or single parameters can be modified after creating the model.

:::{note}
When modifying the geometry using the constructor, only complete surfaces can be modified.
:::

In [3]:
default_navarro_geometry = visisipy.NavarroGeometry()

# Change the thickness of the cornea front surface
default_navarro_geometry.cornea_front.thickness = 1

# Alternatively, specify a new cornea front surface upon model creation
modified_navarro_geometry = visisipy.NavarroGeometry(
    cornea_front=StandardSurface(radius=7.72, asphericity=-0.26, thickness=1),
)

### Easy geometry customization with {py:func}`create_geometry <visisipy.models.geometry.create_geometry>`

The easiest way to create a custom geometry is to use the {py:func}`create_geometry <visisipy.models.geometry.create_geometry>` function.
Using this function, you can specify the geometry using a set of clinically relevant parameters, such as the cornea radius, lens thickness, and axial length.
Parameters that are not specified will be taken from a standard geometry, which is the Navarro geometry by default.
The base geometry can be changed using the `base` parameter of {py:func}`create_geometry <visisipy.models.geometry.create_geometry>`.

As an example, a custom geometry based on the Navarro geometry can be created like this:

In [4]:
myopic_geometry = visisipy.create_geometry(axial_length=26, cornea_front_radius=8.0, lens_thickness=4.0)

### Convenience properties

{py:class}`EyeGeometry <visisipy.models.geometry.EyeGeometry>` objects provide some convenience properties to access clinically relevant parameters.
Note that these properties are calculated from the surface definitions and can therefore not be set directly.

In [5]:
print(f"""
Cornea thickness       : {default_navarro_geometry.cornea_thickness} mm,
Anterior chamber depth : {default_navarro_geometry.anterior_chamber_depth} mm,
Lens thickness         : {default_navarro_geometry.lens_thickness} mm,
Vitreous chamber depth : {default_navarro_geometry.vitreous_thickness} mm,
Axial length           : {default_navarro_geometry.axial_length} mm,
""")


Cornea thickness       : 1 mm,
Anterior chamber depth : 3.05 mm,
Lens thickness         : 4.0 mm,
Vitreous chamber depth : 16.3203 mm,
Axial length           : 24.3703 mm,



## Eye materials

The material properties of the different parts of the eye can be specified using {py:class}`EyeMaterials <visisipy.models.materials.EyeMaterials>`.
This class bundles the material properties of the cornea, aqueous humour, lens, and vitreous humour.
For each of these structures, the material can either be a {py:class}`MaterialModel <visisipy.models.materials.MaterialModel>` or a string.
If a string is used, it should be a valid material name in the selected backend.

{py:class}`MaterialModel <visisipy.models.materials.MaterialModel>`s allow to specify the refractive index, Abbe number, and partial dispersion of the material.
The partial dispersion is only used in the OpticStudio backend and is ignored in other backends.

### Predefined material models

Visisipy provides a number of predefined material models.
Most importantly, the {py:class}`NavarroMaterials <visisipy.models.materials.NavarroMaterials>` class provides the material properties of the Navarro eye model.
This model has been fitted to the refractive indices reported by Escudero-Sanz and Navarro (1999) and works well for all visible wavelengths.
The model accuracy is slightly lower than the specific models for each wavelength, but it is still a good approximation.

Furthermore, material models are provided for all separate wavelengths as reported by Escudero-Sanz and Navarro (1999):

- {py:class}`NavarroMaterials458 <visisipy.models.materials.NavarroMaterials458>`: 458 nm
- {py:class}`NavarroMaterials543 <visisipy.models.materials.NavarroMaterials543>`: 543 nm
- {py:class}`NavarroMaterials589 <visisipy.models.materials.NavarroMaterials589>`: 589.3 nm
- {py:class}`NavarroMaterials633 <visisipy.models.materials.NavarroMaterials633>`: 632.8 nm

:::{attention}
The models for specific wavelengths have a constant refractive index.
It is possible to use them with different wavelengths, but this will result in inaccurate results.
:::

## Building the model

Let's bring it all together!
After defining the eye model in terms of its geometry and materials, the model can be built in the backend using its {py:meth}`build <visisipy.models.EyeModel.build>` method.

In [6]:
final_model = visisipy.EyeModel(
    geometry=default_navarro_geometry, materials=visisipy.models.materials.NavarroMaterials458()
)
final_model.build()

If a backend has not yet been initialized, the default backend will be initialized and used to build the model.
You can read more about that in the next chapter.